# Sampled item evaluation protocol

This notebook aims to reproduce the result for ML1M & Pinterest in the paper ["Revisiting the Performance of iALS on Item Recommendation Benchmarks"](https://arxiv.org/abs/2110.14037). On these two datasets, for each user, we try to rank 1 held-out positive (actually touched by the user) item over 100 randomly selected negative (untouched) items.

Since the protocol is widely used for recsys benchmarking after the [NeuMF paper](https://arxiv.org/abs/1708.05031), below we see how we can measure the recommenders' performance following it. Note, however, there is [a study](https://dl.acm.org/doi/10.1145/3394486.3403226) which asserts that this ranking metric may not be a good indicator for recommender performance.

In [1]:
from irspack import Evaluator, IALSRecommender, df_to_sparse, split_last_n_interaction_df
from irspack.dataset.neu_mf import NeuMFML1MDownloader, NeuMFMPinterestDownloader
import numpy as np
import pandas as pd


# Either ml-1m or pinterest
DATA_TYPE = 'ml-1m'
assert DATA_TYPE in ['ml-1m', 'pinterest']

USER = 'user_id'
ITEM = 'item_id'
TIME = 'timestamp'

if DATA_TYPE == 'ml-1m':
    dm = NeuMFML1MDownloader()
else:
    dm = NeuMFMPinterestDownloader()

## Read the train & test dataset

The train set is a usual user/item interaction dataframe.

In [2]:
train, test = dm.read_train_test()
train.head()

,user_id,item_id,rating,timestamp
0,0,32,4,2001-01-06 23:38:50
1,0,34,4,2001-01-06 23:38:50
2,0,4,5,2001-01-06 23:38:11
3,0,35,4,2001-01-06 23:38:11
4,0,30,4,2001-01-06 23:38:11


In [3]:
item_list = sorted(list(set(train['item_id'])))
item_set = set(item_list)

## Create validation data

Split `train` into train (`tt`) & validation (`tv`) pair.

The validation data is created in the same way as the test set.

In [4]:
g=train.groupby('user_id')['item_id']
user_id_vs_interacted_items = g.agg(set).to_dict()

rng = np.random.default_rng(0)

# tv is users' last interaction with item.
tt, tv = split_last_n_interaction_df(train, USER, timestamp_column=TIME, n_heldout=1)
tv['positive'] = True
dfs = []
for user_id in tv[USER]:
    items_not_interacted = list(item_set - user_id_vs_interacted_items[user_id])
    negatives = rng.choice(items_not_interacted, size=100, replace=False)
    dfs.append(pd.DataFrame({USER: user_id, ITEM: negatives}))
valid = pd.concat(dfs)
valid['positive'] = False
valid = pd.concat([valid, tv[[USER, ITEM, 'positive']]]).sort_values([USER, 'positive'])

The validation dataframe has an extra column to indicate the positivity of the pair.

In [5]:
valid.head()

,user_id,item_id,positive
0,0,1014,False
1,0,131,False
2,0,1281,False
3,0,2669,False
4,0,372,False


Let us convert the data frame into sparse matrix.

In [6]:
X_tt, tt_users, _ = df_to_sparse(tt, USER, ITEM, item_ids=item_list)
X_tv_gt, _, __ = df_to_sparse(valid[valid['positive']], USER, ITEM, user_ids=tt_users, item_ids=item_list)
X_tv_recommendable, _, __ = df_to_sparse(valid, USER, ITEM, user_ids=tt_users, item_ids=item_list)


- Non-zeroes in `X_truth` indicate the positive pair location.
- Non-zeroes in `X_recommendable` are positive & randomly seledted negative pairs.

In the parameter tuning procedure, I found that too eary start of pruning harms the final quality of recommendation.
We can control the parameters of pruning by explicitly providing `optuna.Study`.

In [7]:
validation_evaluator = Evaluator(X_tv_gt, per_user_recommendable_items=X_tv_recommendable, cutoff=10)
best_parameter, validation_recoder = IALSRecommender.tune(
    X_tt, validation_evaluator, fixed_params=dict(n_components=192),
    n_trials=40, random_seed=0, prunning_n_startup_trials=20
)

[I 2022-06-22 23:45:55,197] A new study created in memory with name: no-name-dc21145c-edc2-4822-9a04-74f5c26a98f2
[IRSPACK:I 2022-06-22 23:45:55,198] Start parameter search for IALSRecommender
/home/tomoki/.pyenv/versions/3.10.0/envs/main/lib/python3.10/site-packages/optuna-3.0.0a2-py3.10.egg/optuna/trial/_trial.py:167: FutureWarning: LogUniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~optuna.distributions.FloatDistribution` instead.
  distribution = LogUniformDistribution(low=low, high=high)
[IRSPACK:I 2022-06-22 23:45:55,200] Trial 0:
[IRSPACK:I 2022-06-22 23:45:55,200] parameter = {'alpha0': 0.07272959662822967, 'reg': 0.013981961408994055, 'n_components': 192}


[IRSPACK:I 2022-06-22 23:45:58,153] Config 0 obtained the following scores: {'appeared_item': 2533.0, 'entropy': 7.266555221504073, 'gini_index': 0.71171690172071, 'hit': 0.7433774834437086, 'map': 0.355461867970146, 'n_items': 3704.0, 'ndcg': 0.4471810842792405, 'precision': 0.07433774834437087, 'recall': 0.7433774834437086, 'total_user': 6040.0, 'valid_user': 6040.0} within 2.954025 seconds.
[I 2022-06-22 23:45:58,154] Trial 0 finished with value: -0.4471810842792405 and parameters: {'alpha0': 0.07272959662822967, 'reg': 0.013981961408994055}. Best is trial 0 with value: -0.4471810842792405.
[IRSPACK:I 2022-06-22 23:45:58,155] Trial 1:
[IRSPACK:I 2022-06-22 23:45:58,156] parameter = {'alpha0': 0.09949933190234594, 'reg': 0.004311710058685493, 'n_components': 192}


[IRSPACK:I 2022-06-22 23:45:59,846] Config 1 obtained the following scores: {'appeared_item': 2815.0, 'entropy': 7.497205313232185, 'gini_index': 0.6322773393360319, 'hit': 0.7228476821192052, 'map': 0.3569180332177021, 'n_items': 3704.0, 'ndcg': 0.44372159292884533, 'precision': 0.07228476821192055, 'recall': 0.7228476821192052, 'total_user': 6040.0, 'valid_user': 6040.0} within 1.691079 seconds.
[I 2022-06-22 23:45:59,847] Trial 1 finished with value: -0.44372159292884533 and parameters: {'alpha0': 0.09949933190234594, 'reg': 0.004311710058685493}. Best is trial 0 with value: -0.4471810842792405.
[IRSPACK:I 2022-06-22 23:45:59,848] Trial 2:
[IRSPACK:I 2022-06-22 23:45:59,848] parameter = {'alpha0': 0.03515207314851792, 'reg': 0.008663279761354562, 'n_components': 192}


[IRSPACK:I 2022-06-22 23:46:01,943] Config 2 obtained the following scores: {'appeared_item': 2725.0, 'entropy': 7.334784184071107, 'gini_index': 0.690905169639409, 'hit': 0.7370860927152317, 'map': 0.35466893987175446, 'n_items': 3704.0, 'ndcg': 0.44506594904049857, 'precision': 0.0737086092715232, 'recall': 0.7370860927152317, 'total_user': 6040.0, 'valid_user': 6040.0} within 2.095288 seconds.
[I 2022-06-22 23:46:01,944] Trial 2 finished with value: -0.44506594904049857 and parameters: {'alpha0': 0.03515207314851792, 'reg': 0.008663279761354562}. Best is trial 0 with value: -0.4471810842792405.
[IRSPACK:I 2022-06-22 23:46:01,945] Trial 3:
[IRSPACK:I 2022-06-22 23:46:01,946] parameter = {'alpha0': 0.03811542171376733, 'reg': 0.04734989304499478, 'n_components': 192}


[IRSPACK:I 2022-06-22 23:46:05,537] Config 3 obtained the following scores: {'appeared_item': 1849.0, 'entropy': 6.895916659778797, 'gini_index': 0.8015630408507717, 'hit': 0.6220198675496689, 'map': 0.2712566356564702, 'n_items': 3704.0, 'ndcg': 0.3535569763590235, 'precision': 0.062201986754966895, 'recall': 0.6220198675496689, 'total_user': 6040.0, 'valid_user': 6040.0} within 3.591930 seconds.
[I 2022-06-22 23:46:05,538] Trial 3 finished with value: -0.3535569763590235 and parameters: {'alpha0': 0.03811542171376733, 'reg': 0.04734989304499478}. Best is trial 0 with value: -0.4471810842792405.
[IRSPACK:I 2022-06-22 23:46:05,539] Trial 4:
[IRSPACK:I 2022-06-22 23:46:05,539] parameter = {'alpha0': 0.8097026316647072, 'reg': 0.0014135935551752314, 'n_components': 192}


...

[I 2022-06-22 23:46:50,988] Trial 35 pruned. 
[IRSPACK:I 2022-06-22 23:46:50,997] Trial 36:
[IRSPACK:I 2022-06-22 23:46:50,998] parameter = {'alpha0': 0.11884551336807544, 'reg': 0.0019010987778407956, 'n_components': 192}


[I 2022-06-22 23:46:51,701] Trial 36 pruned. 
[IRSPACK:I 2022-06-22 23:46:51,710] Trial 37:
[IRSPACK:I 2022-06-22 23:46:51,710] parameter = {'alpha0': 0.07794758005102936, 'reg': 0.014163842943737471, 'n_components': 192}


[I 2022-06-22 23:46:51,982] Trial 37 pruned. 
[IRSPACK:I 2022-06-22 23:46:51,990] Trial 38:
[IRSPACK:I 2022-06-22 23:46:51,991] parameter = {'alpha0': 0.04587692461736193, 'reg': 0.007922630332602817, 'n_components': 192}


[IRSPACK:I 2022-06-22 23:46:54,098] Config 38 obtained the following scores: {'appeared_item': 2765.0, 'entropy': 7.373546757550214, 'gini_index': 0.6780582831519174, 'hit': 0.7379139072847682, 'map': 0.35807349153789547, 'n_items': 3704.0, 'ndcg': 0.44793374267887937, 'precision': 0.0737913907284768, 'recall': 0.7379139072847682, 'total_user': 6040.0, 'valid_user': 6040.0} within 2.116132 seconds.
[I 2022-06-22 23:46:54,100] Trial 38 finished with value: -0.44793374267887937 and parameters: {'alpha0': 0.04587692461736193, 'reg': 0.007922630332602817}. Best is trial 29 with value: -0.45715350596349286.
[IRSPACK:I 2022-06-22 23:46:54,109] Trial 39:
[IRSPACK:I 2022-06-22 23:46:54,109] parameter = {'alpha0': 0.3323156467701052, 'reg': 0.0028721481496044128, 'n_components': 192}


[IRSPACK:I 2022-06-22 23:46:55,595] Config 39 obtained the following scores: {'appeared_item': 2624.0, 'entropy': 7.4281180751395715, 'gini_index': 0.6569347617753494, 'hit': 0.7299668874172185, 'map': 0.3592799327236413, 'n_items': 3704.0, 'ndcg': 0.44715632826157664, 'precision': 0.07299668874172188, 'recall': 0.7299668874172185, 'total_user': 6040.0, 'valid_user': 6040.0} within 1.494015 seconds.
[I 2022-06-22 23:46:55,596] Trial 39 finished with value: -0.44715632826157664 and parameters: {'alpha0': 0.3323156467701052, 'reg': 0.0028721481496044128}. Best is trial 29 with value: -0.45715350596349286.


In [8]:
X_train_all, user_ids, _ = df_to_sparse(train, USER, ITEM, item_ids=item_list)
X_test_gt, _, __ = df_to_sparse(test[test["positive"]], USER, ITEM, user_ids=user_ids, item_ids=item_list)
X_test_recommendable, _, __ = df_to_sparse(test, USER, ITEM, user_ids=user_ids, item_ids=item_list)

NDCG@10/HIT@10 is similar to that reported in the reference.

In [10]:
Evaluator(X_test_gt, per_user_recommendable_items=X_test_recommendable, cutoff=10).get_score(
    IALSRecommender(X_train_all, **best_parameter).learn()
)

{'appeared_item': 2714.0,
 'entropy': 7.405280488124141,
 'gini_index': 0.6666785842293266,
 'hit': 0.7310367671414376,
 'map': 0.36564331982102766,
 'n_items': 3704.0,
 'ndcg': 0.45213014176363536,
 'precision': 0.07310367671414376,
 'recall': 0.7310367671414376,
 'total_user': 6040.0,
 'valid_user': 6038.0}